In [ ]:
 pip install kagglehub[pandas-datasets] 

# FORECAST MUMBAI TEMPERATURE WITH DATA SET EXIST

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "rainfall.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "vikramamin/arima-time-series-mumbai-temperature",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

## drop others keep only time and temperature because using model is arima 

In [ ]:
df.columns

In [ ]:
# List of columns to drop
columns_to_drop = ['dew', 'humidity', 'sealevelpressure', 'winddir', 
                   'solarradiation', 'windspeed', 'precipprob', 'preciptype']

# Drop the columns (if they exist in the DataFrame)
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

print("DataFrame after dropping columns:")
print(df.head())

### Convert 'datetime' column to datetime type (if not already) and sort by index

In [ ]:
import pandas as pd

# Convert 'datetime' column with correct format (DD-MM-YYYY)
df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y')

# Set as index and sort
df = df.set_index('datetime').sort_index()

# Verify
print("DataFrame with datetime index:")
print(df.head())
print("\nIndex info:")
print(df.index)

### check stationarity 

In [ ]:
#drop null values in temperature column
ts_data = df['temp'].dropna()  # Ensure no NaN values



### import adfuller from statmodels

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Perform ADF Test
def adf_test(series):
    result = adfuller(series, autolag='AIC')  # Automatic lag selection
    print('--- Augmented Dickey-Fuller Test Results ---')
    print(f'Test Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.4f}')
    
    if result[1] <= 0.05:
        print("✅ **Conclusion**: Reject the null hypothesis (Data is stationary)**")
    else:
        print("❌ **Conclusion**: Fail to reject the null hypothesis (Data is non-stationary)**")

# 4. Run the test
adf_test(ts_data)

### Auto-Regressive (AR) Model - Mathematical Formulation

In [ ]:
import numpy as np

def ar_fit(y, p):
    """
    Fit AR(p) model using Yule-Walker equations.
    
    Parameters:
    y (array): Time series data (stationary)
    p (int): Order of AR model
    
    Returns:
    phi (array): AR coefficients [phi_1, ..., phi_p]
    c (float): Constant term
    var_epsilon (float): Variance of error term
    """
    
    n = len(y)
    y_mean = np.mean(y)
    y_centered = y - y_mean  # Center the data
    
    # 1. Compute autocovariance function R(k)
    R = [np.sum(y_centered[t:] * y_centered[:n-t]) / n for t in range(p+1)]
    
    # 2. Formulate Yule-Walker equations: R = Gamma * phi
    Gamma = np.zeros((p, p))
    for i in range(p):
        for j in range(p):
            Gamma[i,j] = R[abs(i-j)]
    
    r = np.array(R[1:p+1])
    
    # 3. Solve for phi: phi = Gamma^{-1} * r
    phi = np.linalg.solve(Gamma, r)
    
    # 4. Compute constant (c) and error variance
    c = y_mean * (1 - np.sum(phi))
    var_epsilon = R[0] - np.dot(phi, r)
    
    return phi, c, var_epsilon

# Example usage:
# Assuming df['temp'] is your time series data
y = df['temp'].values  # Convert to numpy array
p = 2  # AR(2) model

phi, c, var_epsilon = ar_fit(y, p)

print(f"AR Coefficients (phi_1 to phi_{p}): {phi}")
print(f"Constant term (c): {c:.4f}")
print(f"Error variance (σ²_ε): {var_epsilon:.4f}")

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg

# Example: Fit AR(2) model
model = AutoReg(df['temp'], lags=2)  # AR(2)
results = model.fit()
print(results.summary())

# Extract AR coefficients (phi_1, phi_2)
print("\nAR Coefficients:", results.params[1:])  # Exclude intercept (c)

In [ ]:
import numpy as np

def ma_fit(y, q, max_iter=100, tol=1e-6):
    """
    Robust MA(q) model fitting that handles all edge cases.
    
    Parameters:
    y (array): Time series data (stationary)
    q (int): Order of MA model
    max_iter (int): Maximum iterations
    tol (float): Convergence tolerance
    
    Returns:
    theta (array): MA coefficients [theta_1, ..., theta_q]
    mu (float): Mean of the process
    sigma2 (float): Variance of residuals
    """
    n = len(y)
    if n <= q:
        raise ValueError(f"Time series length {n} must be greater than MA order q={q}")
    
    mu = np.mean(y)
    theta = np.zeros(q)
    eps = np.zeros(n)
    
    prev_sse = float('inf')
    
    for iteration in range(max_iter):
        # Compute errors
        for t in range(n):
            if t < q:
                eps[t] = y[t] - mu
            else:
                # Get previous q errors safely
                prev_eps = eps[max(0, t-q):t][::-1]
                if len(prev_eps) < q:
                    prev_eps = np.pad(prev_eps, (0, q - len(prev_eps)))
                eps[t] = y[t] - mu - np.dot(theta, prev_eps)
        
        # Prepare design matrix
        X = []
        y_target = []
        for t in range(q, n):
            prev_eps = eps[t-1:t-q-1:-1]
            if len(prev_eps) < q:
                prev_eps = np.pad(prev_eps, (0, q - len(prev_eps)))
            X.append(prev_eps)
            y_target.append(y[t] - mu)
        
        X = np.array(X)
        y_target = np.array(y_target)
        
        # Solve least squares
        theta_new, res, _, _ = np.linalg.lstsq(X, y_target, rcond=None)
        
        # Check convergence
        current_sse = np.sum(res)
        if iteration > 0 and abs(prev_sse - current_sse) < tol:
            break
            
        theta = theta_new
        prev_sse = current_sse
    
    # Calculate residual variance
    sigma2 = np.var(eps[q:], ddof=q) if q < n else np.var(eps)
    
    return theta, mu, sigma2

# Example usage:
y = df['temp'].values  # Ensure this is stationary
q = 1  # MA(1) model

try:
    theta, mu, sigma2 = ma_fit(y, q)
    print(f"MA Coefficients (theta_1 to theta_{q}): {theta}")
    print(f"Process mean (μ): {mu:.4f}")
    print(f"Residual variance (σ²): {sigma2:.4f}")
    
    # Prediction example
    eps = y - mu
    for t in range(q, len(y)):
        prev_eps = eps[max(0, t-q):t][::-1]
        if len(prev_eps) < q:
            prev_eps = np.pad(prev_eps, (0, q - len(prev_eps)))
        eps[t] -= np.dot(theta, prev_eps)
    
    last_eps = eps[-q:] if q > 0 else []
    next_pred = mu + (np.dot(theta, last_eps) if q > 0 else 0)
    print(f"Next predicted value: {next_pred:.4f}")
    
except ValueError as e:
    print(f"Error: {e}")

In [ ]:
pip install scipy numpy matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import toeplitz, solve_toeplitz

def plot_acf_pacf(y, lags=20):
    """
    Robust ACF/PACF plotting with error handling
    
    Parameters:
    y (array): Time series data (should be stationary)
    lags (int): Number of lags to display (max 1/5 of data length)
    """
    n = len(y)
    lags = min(lags, n//5)  # Ensure we don't request too many lags
    if n < 5:
        raise ValueError("Time series too short (need at least 5 observations)")
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    conf = 1.96/np.sqrt(n)
    
    # ==================== ACF Plot ==================== #
    acf = [1.0]
    for i in range(1, lags+1):
        if i >= n:
            acf.append(np.nan)
        else:
            corr = np.corrcoef(y[:-i], y[i:])[0,1]
            acf.append(corr if not np.isnan(corr) else 0)
    
    ax1.stem(range(lags+1), acf, linefmt='b-', markerfmt='bo', basefmt='k-')
    ax1.axhspan(-conf, conf, color='lightgray', alpha=0.5)
    ax1.set_title('Autocorrelation Function (ACF)')
    
    # ==================== PACF Plot ==================== #
    pacf = [1.0]
    for k in range(1, lags+1):
        try:
            r = [np.corrcoef(y[:-i], y[i:])[0,1] for i in range(1, k+1)]
            R = toeplitz(r[:-1])
            phi = solve_toeplitz(R, r[1:])
            pacf.append(phi[-1] if len(phi) > 0 else np.nan)
        except:
            pacf.append(np.nan)
    
    ax2.stem(range(lags+1), pacf, linefmt='g-', markerfmt='go', basefmt='k-')
    ax2.axhspan(-conf, conf, color='lightgray', alpha=0.5)
    ax2.set_title('Partial Autocorrelation Function (PACF)')
    
    plt.tight_layout()
    plt.show()

# Example usage:
if 'df' in globals() and 'temp' in df.columns:
    y = df['temp'].values
    plot_acf_pacf(y, lags=20)
else:
    print("Please ensure 'df' exists and has a 'temp' column")

## The models shows AR(2) model PACF cuts off after lag 2 → AR(2).
## The ACF model came tend to zero gratually and it taped off then MA(0)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# Fit AR(1)
model_ar1 = ARIMA(y, order=(1, 0, 0)).fit()
# Fit AR(2)
model_ar2 = ARIMA(y, order=(2, 0, 0)).fit()
print("AR(1) AIC:", model_ar1.aic)
print("AR(2) AIC:", model_ar2.aic)  # Lower AIC → Better model

## AIC: 4339.03555306611 is minimum one use ARIMA(2,0,0) model for that .

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Load your data (replace with your actual data)
y = df['temp'].values  

# Fit ARIMA(2,0,0) model
model = ARIMA(y, order=(2, 0, 0))  # p=2, d=0, q=0
results = model.fit()

# Print model summary
print(results.summary())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.gofplots import qqplot
from sklearn.metrics import mean_squared_error
import warnings

# Suppress only the specific covariance matrix warning
warnings.filterwarnings('ignore', message='Quasi-maximum likelihood covariance matrix')

# Load your data
y = df['temp'].values  

# Fit ARIMA(2,0,0) model with optimized settings
model = ARIMA(y, order=(2, 0, 0), trend='c')  # Explicit constant term
results = model.fit(method='innovations_mle',  # Best estimation method
                   cov_type='robust')         # Robust standard errors

# Print clean model summary
with pd.option_context('display.float_format', '{:.4f}'.format):
    print(results.summary())
    print("\nKey Interpretation:")
    print(f"- AR(1) coefficient: {results.arparams[0]:.4f} (p={results.pvalues[0]:.4f})")
    print(f"- AR(2) coefficient: {results.arparams[1]:.4f} (p={results.pvalues[1]:.4f})")
    print(f"- Constant term: {results.params[0]:.4f}")
    print(f"- AIC: {results.aic:.2f}, BIC: {results.bic:.2f}")


In [ ]:
# Split data into train/test (e.g., 80/20)
train_size = int(len(y) * 0.8)
train, test = y[:train_size], y[train_size:]

# Fit model on training data
model_train = ARIMA(train, order=(2, 0, 0)).fit()

# Forecast test period
forecast_test = model_train.get_forecast(steps=len(test))
forecast_values = forecast_test.predicted_mean

# Calculate error metrics
mse = mean_squared_error(test, forecast_values)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(test - forecast_values))

print(f"Test MSE: {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")

In [ ]:
# Refit on full data (if errors are acceptable)
final_model = ARIMA(y, order=(2, 0, 0)).fit()

# Continue forecasting beyond initial forecast
new_forecast_steps = 10
new_forecast = final_model.get_forecast(steps=new_forecast_steps)
new_forecast_mean = new_forecast.predicted_mean

print("Extended Forecast:\n", new_forecast_mean)

In [ ]:
residuals = results.resid
plt.figure(figsize=(12, 4))
plt.plot(residuals)
plt.title("Residuals Plot")
plt.show()

# ACF of residuals (should show no significant lags)
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(residuals, lags=20)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Load your data (replace with your actual data)
y = df['temp'].values  

# Fit ARIMA(2,0,0) model
model = ARIMA(y, order=(2, 0, 0))  # p=2, d=0, q=0
results = model.fit()

# Print model summary
print(results.summary())

# Forecast next 5 steps
forecast_steps = 5
forecast = results.get_forecast(steps=forecast_steps)
forecast_mean = forecast.predicted_mean
conf_int = forecast.conf_int()  # Returns numpy array

print("Forecasted Values:\n", forecast_mean)
print("Confidence Intervals:\n", conf_int)

# Plot historical data + forecast correctly
plt.figure(figsize=(12, 6))
plt.plot(y, label='Historical Data')

# Corrected plotting - use numpy array indexing
forecast_index = np.arange(len(y), len(y) + forecast_steps)
plt.plot(forecast_index, forecast_mean, 'r--', label='Forecast')
plt.fill_between(forecast_index,
                 conf_int[:, 0],  # Use numpy indexing
                 conf_int[:, 1],  # instead of .iloc
                 color='pink', alpha=0.3)

plt.legend()
plt.title("ARIMA(2,0,0) Forecast")
plt.show()

# Residual analysis
residuals = results.resid
plt.figure(figsize=(12, 4))
plt.plot(residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title("Model Residuals")
plt.show()

# ACF of residuals
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(residuals, lags=20)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.gofplots import qqplot  # Only qqplot is available here
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_squared_error

#  Forecast test period with confidence intervals
forecast = results.get_forecast(steps=len(test))
forecast_mean = forecast.predicted_mean
conf_int = forecast.conf_int(alpha=0.05)  # 95% CI

#  Calculate error metrics
mse = mean_squared_error(test, forecast_mean)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(test - forecast_mean))

print("Model Performance Metrics:")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}\n")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.gofplots import qqplot
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_squared_error

# Load your data
y = df['temp'].values  

# 1. Split data into train/test (80/20)
train_size = int(len(y) * 0.8)
train, test = y[:train_size], y[train_size:]

# 2. Fit ARIMA(2,0,0) model
model = ARIMA(train, order=(2, 0, 0))
results = model.fit()

# 3. Forecast test period with confidence intervals
forecast = results.get_forecast(steps=len(test))
forecast_mean = forecast.predicted_mean
conf_int = forecast.conf_int(alpha=0.05)  # 95% CI

# 4. Calculate error metrics
mse = mean_squared_error(test, forecast_mean)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(test - forecast_mean))

print("Model Performance Metrics:")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}\n")

# 5. Create comprehensive diagnostic plots
plt.figure(figsize=(15, 12))

# 5a. Forecast vs Actual plot (FIXED)
plt.subplot(3, 2, 1)
plt.plot(np.arange(len(train)), train, 'b-', label='Training Data')
plt.plot(np.arange(len(train), len(train)+len(test)), 
         test, 'g-', label='Actual Test')  # Plot test directly without concatenation
plt.plot(np.arange(len(train), len(train)+len(test)), 
         forecast_mean, 'r--', label='Forecast')
plt.fill_between(np.arange(len(train), len(train)+len(test)),
                 conf_int[:, 0], conf_int[:, 1], 
                 color='pink', alpha=0.3, label='95% CI')
plt.title('Forecast vs Actual')
plt.legend()
plt.grid(True)

# 5b. Normal QQ plot
plt.subplot(3, 2, 2)
qqplot(results.resid, line='q', ax=plt.gca())
plt.title('Normal Q-Q Plot')

# 5c. Normal PP plot (using scipy.stats)
plt.subplot(3, 2, 3)
stats.probplot(results.resid, plot=plt)
plt.title('Normal P-P Plot')

# 5d. Correlogram (ACF of residuals)
plt.subplot(3, 2, 4)
plot_acf(results.resid, lags=20, ax=plt.gca())
plt.title('Residual Correlogram')

# 5e. Histogram with density estimate
plt.subplot(3, 2, 5)
residuals = results.resid[2:]  # Skip first two due to AR(2)
plt.hist(residuals, bins=30, density=True, alpha=0.6, color='g')
resid_min, resid_max = residuals.min(), residuals.max()
x = np.linspace(resid_min, resid_max, 100)
plt.plot(x, stats.norm.pdf(x, np.mean(residuals), np.std(residuals)), 'r-')
plt.title('Residual Histogram & Normal Density')
plt.xlabel('Residuals')

# 5f. Forecast margin plot
plt.subplot(3, 2, 6)
forecast_margin = (conf_int[:, 1] - conf_int[:, 0]) / 2
plt.plot(np.arange(len(test)), forecast_margin, 'b-o')
plt.title('Forecast Margin (Half CI Width)')
plt.xlabel('Forecast Step')
plt.ylabel('Margin')
plt.grid(True)

plt.tight_layout()
plt.show()



In [ ]:
# Print forecast table with margins
forecast_df = pd.DataFrame({
    'Actual': test,
    'Forecast': forecast_mean,
    'Lower_CI': conf_int[:, 0],
    'Upper_CI': conf_int[:, 1],
    'Margin': forecast_margin
}, index=pd.RangeIndex(len(train), len(train)+len(test)))

print("\nForecast Results with Margins:")
print(forecast_df.round(4))

## DO AUTO ARIMA for the get best fitted model

In [ ]:
pip install pmdarima

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pmdarima import auto_arima
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Load your data
y = df['temp'].values  

# Run Auto-ARIMA to find optimal model
model = auto_arima(
    y,
    start_p=0,
    start_q=0,
    max_p=3,
    max_q=3,
    d=None,
    seasonal=False,
    trace=True,
    error_action='ignore',
    stepwise=True,
    suppress_warnings=True
)

# Output the best model
print("="*50)
print("Best ARIMA Model Summary")
print("="*50)
print(model.summary())

# Generate forecasts for next 10 periods
forecast_steps = 10
forecast, conf_int = model.predict(
    n_periods=forecast_steps,
    return_conf_int=True,
    alpha=0.05  # 95% confidence intervals
)

# Create time indices for plotting
train_index = np.arange(len(y))
forecast_index = np.arange(len(y), len(y) + forecast_steps)

# Plot historical data and forecast
plt.figure(figsize=(14, 7))
plt.plot(train_index, y, 'b-', label='Historical Data')
plt.plot(forecast_index, forecast, 'r--', label='Forecast')
plt.fill_between(
    forecast_index,
    conf_int[:, 0],
    conf_int[:, 1],
    color='pink',
    alpha=0.3,
    label='95% Confidence Interval'
)
plt.title(f'ARIMA{model.order} Forecast - Next {forecast_steps} Periods', fontsize=14)
plt.xlabel('Time Index', fontsize=12)
plt.ylabel('Temperature', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

# Enhanced diagnostics
print("\n" + "="*50)
print("Model Diagnostics")
print("="*50)

# 1. Residual Analysis
residuals = model.resid()
plt.figure(figsize=(14, 10))

# Residuals plot
plt.subplot(2, 2, 1)
plt.plot(residuals)
plt.axhline(0, color='r', linestyle='--')
plt.title('Residuals Over Time')
plt.xlabel('Time Index')
plt.ylabel('Residuals')
plt.grid(True)

# Histogram and Q-Q plot
plt.subplot(2, 2, 2)
plt.hist(residuals, bins=30, density=True, alpha=0.6)
x = np.linspace(residuals.min(), residuals.max(), 100)
plt.plot(x, stats.norm.pdf(x, residuals.mean(), residuals.std()), 'r-')
plt.title('Residual Distribution')
plt.xlabel('Residuals')

# ACF of residuals
plt.subplot(2, 2, 3)
plot_acf(residuals, lags=20, ax=plt.gca())
plt.title('Residual Autocorrelation')

# Forecast error analysis
plt.subplot(2, 2, 4)
forecast_errors = conf_int[:, 1] - conf_int[:, 0]
plt.plot(forecast_index, forecast_errors, 'bo-')
plt.title('Forecast Confidence Interval Width')
plt.xlabel('Forecast Step')
plt.ylabel('CI Width')
plt.grid(True)

plt.tight_layout()
plt.show()



In [ ]:
# Forecast details dataframe
forecast_df = pd.DataFrame({
    'Forecast': forecast,
    'Lower_CI': conf_int[:, 0],
    'Upper_CI': conf_int[:, 1],
    'CI_Width': conf_int[:, 1] - conf_int[:, 0]
}, index=forecast_index)

print("\nDetailed Forecast Results:")
print(forecast_df.round(4))

# Model evaluation metrics
print("\nModel Evaluation:")
print(f"AIC: {model.aic():.2f}")
print(f"BIC: {model.bic():.2f}")
print(f"HQIC: {model.hqic():.2f}")

In [ ]:
import numpy as np
import pandas as pd
from pmdarima import auto_arima

# Load your data and fit model (as before)
y = df['temp'].values  
model = auto_arima(y, seasonal=False, trace=True)

# Generate forecast with confidence intervals
forecast_steps = 7  # Next 7 days
forecast, conf_int = model.predict(
    n_periods=forecast_steps,
    return_conf_int=True,
    alpha=0.05
)

# Create date range for forecast period
last_date = df.index[-1]  # Assuming your dataframe has a datetime index
forecast_dates = pd.date_range(
    start=last_date + pd.Timedelta(days=1),
    periods=forecast_steps,
    freq='D'
)

# Calculate margins (half of CI width)
margins = (conf_int[:, 1] - conf_int[:, 0]) / 2

# Create forecast dataframe
forecast_df = pd.DataFrame({
    'Date': forecast_dates.date,  # Just the date part without time
    'Day': forecast_dates.day_name(),
    'Forecasted_Temperature': forecast.round(2),
    'Lower_Bound': conf_int[:, 0].round(2),
    'Upper_Bound': conf_int[:, 1].round(2),
    'Margin': margins.round(2)
})

# Print formatted forecast table
print("\nDaily Temperature Forecast with Confidence Margins")
print("="*60)
print(forecast_df.to_string(index=False))
print("\nKey:")
print("- Margin represents ± variation from forecasted value")
print("- 95% confidence interval: [Lower_Bound, Upper_Bound]")